In [15]:
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
import wget
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import cv2
from sklearn.tree import export_graphviz
from sklearn.neighbors import KNeighborsClassifier as KNC
from statistics import mean
import pydotplus
import pydot
import graphviz 
from io import StringIO
from IPython.display import display, Image
from tqdm.notebook import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import cv2
from sklearn.tree import export_graphviz
from sklearn.neighbors import KNeighborsClassifier as KNC
from statistics import mean
import pydotplus
import pydot
import graphviz 
from io import StringIO
from IPython.display import display, Image
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor

# Download

In [16]:
# PD_data_train = wget.download('https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv')
# PD_data_test = wget.download('https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-test.csv')
# PD_data_desc=wget.download('https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-desc.csv')


In [17]:
train_df = pd.read_csv('PD-data-train.csv', sep=';')
test_df = pd.read_csv('PD-data-test.csv', sep=';')
desc_df = pd.read_csv('PD-data-desc.csv', sep=';')

In [18]:
train_df_newversion_X=train_df.drop(['record_id','default_12m'], axis=1)
train_df_newversion_Y=train_df['default_12m']

In [19]:
# for i in train_df_newversion_X.columns:
#   print(F'{i}:  unique={len(train_df_newversion_X[i].unique())},  isnull={train_df_newversion_X[i].isna().sum()}')

In [20]:
train_df_newversion_X_categorical=train_df_newversion_X[['ul_branch_cnt','ul_strategic_flg','ul_systematizing_flg','ul_staff_range']]
train_df_newversion_X_number=train_df_newversion_X.drop(['ul_staff_range','ul_branch_cnt','ul_strategic_flg','ul_systematizing_flg'], axis=1)


# third way

In [21]:
def impute(df):
    for name in df.select_dtypes("number"):
        df[name] = df[name].fillna(df[name].mean())
        df[name][df[name]==0]=df[name].mean() #можно добавить шум ибо тут явно переобучение
        df[name]=df[name].round(1)
    return df

def categor(df):
    g=df.columns
    df2=pd.DataFrame()
    for name in g:
        one_hot = pd.get_dummies(df[name],prefix=[f'{name}'])
        df = df.drop(name,axis = 1)
        df2 = pd.concat([one_hot,df2],axis=1)
    return df2
train_df_newversion_X_categorical=pd.concat([train_df_newversion_X_categorical,train_df_newversion_X_number[['adr_actual_age','head_actual_age','cap_actual_age']]],axis=1)
print(len(train_df_newversion_X_categorical.columns))
print(train_df_newversion_X_categorical.columns)
train_df_newversion_X_categorical=categor(train_df_newversion_X_categorical)
check_test=train_df_newversion_X_categorical.columns
check=train_df_newversion_X_categorical

7
Index(['ul_branch_cnt', 'ul_strategic_flg', 'ul_systematizing_flg',
       'ul_staff_range', 'adr_actual_age', 'head_actual_age',
       'cap_actual_age'],
      dtype='object')


In [22]:
capital=np.log1p(abs(train_df_newversion_X_number['ul_capital_sum']))
train_df_newversion_X_categorical=pd.concat([train_df_newversion_X_categorical,train_df_newversion_X_number[['bus_age','ul_founders_cnt','ogrn_age']]],axis=1)
train_df_newversion_X_categorical=pd.concat([train_df_newversion_X_categorical,capital], axis=1)

In [23]:
train_df_newversion_X_number=impute(train_df_newversion_X_number)


In [24]:
train_df_newversion_edited=pd.concat([train_df_newversion_X_number,train_df_newversion_X_categorical],axis=1)

In [25]:
train_df_newversion_edited

,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,...,['ul_branch_cnt']_8,['ul_branch_cnt']_10,['ul_branch_cnt']_11,['ul_branch_cnt']_12,['ul_branch_cnt']_17,['ul_branch_cnt']_20,bus_age,ul_founders_cnt,ogrn_age,ul_capital_sum
0,52996000.0,48344000.0,48344000.0,2499198.2,1082262.8,4652000.0,-2608000.0,2044000.0,846000.0,39373.0,...,0,0,0,0,0,0,198,2,136,10.819798
1,81537755.8,76784740.1,73203279.1,2499198.2,1082262.8,4753015.7,-642477.3,4110538.4,418807.1,39373.0,...,0,0,0,0,0,0,256,5,135,9.210440
2,81537755.8,76784740.1,73203279.1,2499198.2,1082262.8,4753015.7,-642477.3,4110538.4,418807.1,39373.0,...,0,0,0,0,0,0,33,2,33,9.210440
3,32049000.0,29568000.0,29568000.0,2499198.2,1082262.8,2481000.0,-260000.0,2221000.0,302000.0,39373.0,...,0,0,0,0,0,0,185,1,139,12.611541
4,16458000.0,14935000.0,14935000.0,2499198.2,1082262.8,1523000.0,-274000.0,1249000.0,538000.0,39373.0,...,0,0,0,0,0,0,220,2,138,9.210440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32390,28264000.0,27911000.0,27911000.0,2499198.2,1082262.8,353000.0,-68000.0,285000.0,57000.0,39373.0,...,0,0,0,0,0,0,71,1,71,9.210440
32391,25017000.0,24381000.0,24381000.0,2499198.2,1082262.8,636000.0,-642477.3,636000.0,418807.1,39373.0,...,0,0,0,0,0,0,168,1,141,13.304687
32392,81537755.8,76784740.1,73203279.1,2499198.2,1082262.8,4753015.7,-642477.3,4110538.4,418807.1,39373.0,...,0,0,0,0,0,0,11,1,11,9.210440
32393,136162000.0,135088000.0,135088000.0,2499198.2,1082262.8,1074000.0,-719000.0,355000.0,267000.0,39373.0,...,0,0,0,0,0,0,70,1,70,9.305741


In [26]:
# X_NonNa=pd.concat([NonNa_num,NonNa_cat],axis=1)

In [27]:
# X_NonNa

In [28]:
# Y_nonNa=train_df_newversion_Y[-train_df_newversion_X_number['ar_revenue'].isna()]

In [29]:
# train_df_newversion_edited=X_NonNa

In [30]:
# train_df_newversion_Y=Y_nonNa

In [ ]:
# train_df_newversion_edited

# Solving


In [ ]:
train_df_newversion_edited

In [ ]:
train_df_newversion_Y

In [ ]:
import sklearn
sklearn.metrics.SCORERS.keys()

In [ ]:

X_train1, X_test1, y_train1, y_test1 = train_test_split(train_df_newversion_edited,
                                                    train_df_newversion_Y,
                                                    random_state=42, shuffle=True)

logit = LogisticRegression(C=1,solver='lbfgs',max_iter=5000)
cv_baseline = cross_val_score(logit, X_train1, y_train1, cv=5, scoring='accuracy').mean()



In [ ]:
cv_baseline

In [ ]:
logit2 = LogisticRegression(C=1,solver='lbfgs',max_iter=5000)
# cv_baseline = cross_val_score(logit2, train_df_newversion_edited, train_df_newversion_Y, cv=5, scoring='accuracy').mean()
# cv_baseline

In [ ]:
logit2.fit(train_df_newversion_edited,train_df_newversion_Y)

In [ ]:
# #clf=LogisticRegressionCV(cv=5,random_state=42).fit(train_df_newversion_edited, train_df_newversion_Y)
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification
# clf = RandomForestClassifier(max_depth=5, random_state=0)
# clf.fit(train_df_newversion_edited, train_df_newversion_Y)

In [ ]:
def Logit_ElasticNet(X_train,y_train):
    
    params = {'C': np.arange(0.01,0.15,0.01), 'l1_ratio': np.linspace(0,0.6,15)}
    log_ElNet = GridSearchCV(LogisticRegression(penalty='elasticnet', solver='saga', random_state=1), params, scoring='accuracy', cv=5)
    log_ElNet.fit(X_train, y_train)

    print('best score:', log_ElNet.best_score_)
    print('best params:', log_ElNet.best_params_)
    return log_ElNet


In [ ]:
log_ElNet2 = Logit_ElasticNet(train_df_newversion_edited,train_df_newversion_Y)

In [ ]:
from sklearn.svm import SVC
def svm(X_train,y_train):
    
    params = {'C': np.arange(120,180,1)}
    svm = GridSearchCV(SVC(), params, scoring='f1', cv=5)
    svm.fit(X_train, y_train)

    print('best score:', svm.best_score_)
    print('best params:', svm.best_params_)
    return svm

svm = svm(train_df_newversion_edited,train_df_newversion_Y)

# test

In [31]:
test_df.columns

Index(['record_id', 'ar_revenue', 'ar_total_expenses', 'ar_sale_cost',
       'ar_selling_expenses', 'ar_management_expenses', 'ar_sale_profit',
       'ar_balance_of_rvns_and_expns', 'ar_profit_before_tax', 'ar_taxes',
       'ar_other_profit_and_losses', 'ar_net_profit', 'ab_immobilized_assets',
       'ab_mobile_current_assets', 'ab_inventory', 'ab_accounts_receivable',
       'ab_other_current_assets', 'ab_cash_and_securities', 'ab_losses',
       'ab_own_capital', 'ab_borrowed_capital', 'ab_long_term_liabilities',
       'ab_short_term_borrowing', 'ab_accounts_payable', 'ab_other_borrowings',
       'bus_age', 'ogrn_age', 'adr_actual_age', 'head_actual_age',
       'cap_actual_age', 'ul_staff_range', 'ul_capital_sum', 'ul_founders_cnt',
       'ul_branch_cnt', 'ul_strategic_flg', 'ul_systematizing_flg'],
      dtype='object')

In [32]:
train_df.columns

Index(['record_id', 'ar_revenue', 'ar_total_expenses', 'ar_sale_cost',
       'ar_selling_expenses', 'ar_management_expenses', 'ar_sale_profit',
       'ar_balance_of_rvns_and_expns', 'ar_profit_before_tax', 'ar_taxes',
       'ar_other_profit_and_losses', 'ar_net_profit', 'ab_immobilized_assets',
       'ab_mobile_current_assets', 'ab_inventory', 'ab_accounts_receivable',
       'ab_other_current_assets', 'ab_cash_and_securities', 'ab_losses',
       'ab_own_capital', 'ab_borrowed_capital', 'ab_long_term_liabilities',
       'ab_short_term_borrowing', 'ab_accounts_payable', 'ab_other_borrowings',
       'bus_age', 'ogrn_age', 'adr_actual_age', 'head_actual_age',
       'cap_actual_age', 'ul_staff_range', 'ul_capital_sum', 'ul_founders_cnt',
       'ul_branch_cnt', 'ul_strategic_flg', 'ul_systematizing_flg',
       'default_12m'],
      dtype='object')

In [33]:
test=test_df.drop(['record_id'], axis=1)


In [34]:
test_categorical=test[['ul_branch_cnt','ul_strategic_flg','ul_systematizing_flg','ul_staff_range']]
test_number=test.drop(['ul_staff_range','ul_branch_cnt','ul_strategic_flg','ul_systematizing_flg'], axis=1)


In [35]:
test_categorical

,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,ul_staff_range
0,0,0,0,[1-100]
1,0,0,0,[1-100]
2,0,0,0,[1-100]
3,0,0,0,[1-100]
4,0,0,0,(100-500]
...,...,...,...,...
195,0,0,0,[1-100]
196,0,0,0,[1-100]
197,0,0,0,[1-100]
198,0,0,0,[1-100]


In [36]:

test_categorical=pd.concat([test_categorical,test_number[['adr_actual_age','head_actual_age','cap_actual_age']]],axis=1)
print(len(test_categorical.columns))
print(test_categorical.columns)

7
Index(['ul_branch_cnt', 'ul_strategic_flg', 'ul_systematizing_flg',
       'ul_staff_range', 'adr_actual_age', 'head_actual_age',
       'cap_actual_age'],
      dtype='object')


In [37]:
c=check_test.tolist()

In [38]:
test_categorical=categor(test_categorical)


In [39]:
l=["['cap_actual_age']_30","['head_actual_age']_30","['adr_actual_age']_30","['ul_strategic_flg']_1",
                             "['ul_branch_cnt']_1",
                             "['ul_branch_cnt']_2",
                             "['ul_branch_cnt']_3",
                             "['ul_branch_cnt']_4",
                             "['ul_branch_cnt']_5",
                             "['ul_branch_cnt']_6",
                             "['ul_branch_cnt']_8",
                             "['ul_branch_cnt']_10",
                             "['ul_branch_cnt']_11",
                             "['ul_branch_cnt']_12",
                             "['ul_branch_cnt']_17",
                             "['ul_branch_cnt']_20"]

In [40]:
for i in l:
    test_categorical[l]=0

In [41]:
len(check_test.tolist())

118

In [42]:
len(test_categorical.columns.tolist())

118

In [43]:
capital=np.log1p(abs(test_number['ul_capital_sum']))
test_categorical=pd.concat([test_categorical,test_number[['bus_age','ul_founders_cnt','ogrn_age']]],axis=1)
test_categorical=pd.concat([test_categorical,capital], axis=1)

In [44]:
test_number=test_number.drop(['bus_age','ul_founders_cnt','ogrn_age','adr_actual_age','head_actual_age','cap_actual_age','ul_capital_sum'],axis=1)

In [45]:
for i in test_number.columns:
    test_number[i]=np.log1p(abs(test_number[i]))

In [46]:
test_number.reset_index()['index']

0        0
1        1
2        2
3        3
4        4
      ... 
195    195
196    196
197    197
198    198
199    199
Name: index, Length: 200, dtype: int64

In [47]:

ide_NonNa=test_number.reset_index()['index'][-test['ar_revenue'].isna()]
ide_YesNa=test_number.reset_index()['index'][test['ar_revenue'].isna()]
Test_NonNa_num=test_number[-test['ar_revenue'].isna()]
Test_YesNa_num=test_number[test['ar_revenue'].isna()]

Test_NonNa_cat=test_categorical[-test['ar_revenue'].isna()]
Test_YesNa_cat=test_categorical[test['ar_revenue'].isna()]

In [48]:
print(len(test_categorical.columns))

122


In [49]:
Test_NonNa_num

,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,...,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings
0,17.481710,17.476839,17.476839,0.000000,0.000000,12.154785,11.856522,12.709877,12.556733,0.0,...,15.084709,0.000000,13.425428,10.757924,13.979329,15.628705,0.000000,14.914123,14.956544,0.0
2,17.807712,17.745943,17.745943,0.000000,0.000000,14.992626,14.060808,14.492021,13.122365,0.0,...,11.835016,0.000000,12.375820,14.198731,15.737567,15.178025,0.000000,0.000000,15.178025,0.0
3,14.459468,14.333710,14.333710,0.000000,0.000000,12.323860,11.385103,11.827744,0.000000,0.0,...,14.204847,0.000000,10.463132,11.827744,15.554397,13.199326,0.000000,13.122365,10.596660,0.0
4,19.234203,19.165875,19.165875,0.000000,0.000000,16.516805,16.358293,14.596669,13.005832,0.0,...,16.784848,0.000000,13.770514,14.368821,16.333852,17.666020,0.000000,16.601447,17.243090,0.0
5,16.485997,16.443290,16.443290,0.000000,0.000000,13.311331,12.899222,13.819504,0.000000,0.0,...,0.000000,0.000000,0.000000,13.819504,14.261478,13.010316,0.000000,0.000000,13.010316,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,17.039255,16.258119,16.258119,0.000000,0.000000,16.426977,14.182236,16.527686,12.509878,0.0,...,15.420338,0.000000,12.460719,16.509530,17.251592,15.519895,0.000000,15.367108,13.565768,0.0
192,17.601880,17.592744,17.592744,0.000000,0.000000,12.901719,9.952325,12.847929,11.561725,0.0,...,15.884270,0.000000,14.912455,12.524530,14.236850,16.689584,0.000000,0.000000,16.689584,0.0
193,15.940523,15.931525,15.931525,0.000000,0.000000,11.225257,9.305741,11.066654,9.952325,0.0,...,13.814511,0.000000,11.982935,10.668979,11.211834,14.606692,0.000000,0.000000,14.606692,0.0
195,16.873607,16.751387,16.602187,14.775244,0.000000,14.711191,11.719948,14.659661,13.087774,0.0,...,14.498103,6.908755,14.745310,14.428532,15.302555,15.719706,14.011078,0.000000,15.519895,0.0


In [51]:
# params = {'C': np.arange(0.01,0.11,0.01), 'l1_ratio': np.linspace(0,1,10)}
# pr_test=pd.DataFrame()
# for i in Test_YesNa_num.columns:  #because i predict number par by categorical now y is learning bas and x is predicting
#     model = RandomForestRegressor(random_state=1).fit(Test_NonNa_cat, Test_NonNa_num[i])
# #     pr=pd.concat([pd.DataFrame({i:model.predict(NonNa_cat).reshape(-1,)}),pr],axis=1)
# #     print(f'{i}: ac_train={model.score(x_train,y_train)},ac_test={model.score(x_test, y_test)}')
#     toTest=pd.concat([pd.DataFrame({i:model.predict(Test_YesNa_cat).reshape(-1,)}),toTest],axis=1)
#     print(i)

In [52]:
ide=pd.concat([ide_YesNa,ide_NonNa],axis=0)
numbers_t=pd.concat([Test_YesNa_num,Test_NonNa_num],axis=0)
numbers_t=impute(numbers_t)
cat_t=pd.concat([Test_YesNa_cat,Test_NonNa_cat],axis=0)
LAST=numbers_t.join(cat_t)

In [53]:
LAST

,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,...,['ul_branch_cnt']_8,['ul_branch_cnt']_10,['ul_branch_cnt']_11,['ul_branch_cnt']_12,['ul_branch_cnt']_17,['ul_branch_cnt']_20,bus_age,ul_founders_cnt,ogrn_age,ul_capital_sum
1,15.6,15.7,15.6,2.3,0.6,13.0,10.8,12.9,7.7,0.9,...,0,0,0,0,0,0,216,1,138,11.512935
6,15.6,15.7,15.6,2.3,0.6,13.0,10.8,12.9,7.7,0.9,...,0,0,0,0,0,0,98,1,98,9.210440
7,15.6,15.7,15.6,2.3,0.6,13.0,10.8,12.9,7.7,0.9,...,0,0,0,0,0,0,43,1,43,9.210440
8,15.6,15.7,15.6,2.3,0.6,13.0,10.8,12.9,7.7,0.9,...,0,0,0,0,0,0,16,1,16,9.210440
9,15.6,15.7,15.6,2.3,0.6,13.0,10.8,12.9,7.7,0.9,...,0,0,0,0,0,0,33,1,33,9.210440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,17.0,16.3,16.3,2.3,0.6,16.4,14.2,16.5,12.5,0.9,...,0,0,0,0,0,0,95,1,95,14.735794
192,17.6,17.6,17.6,2.3,0.6,12.9,10.0,12.8,11.6,0.9,...,0,0,0,0,0,0,60,2,60,9.210440
193,15.9,15.9,15.9,2.3,0.6,11.2,9.3,11.1,10.0,0.9,...,0,0,0,0,0,0,37,1,37,9.210440
195,16.9,16.8,16.6,14.8,0.6,14.7,11.7,14.7,13.1,0.9,...,0,0,0,0,0,0,47,1,47,9.210440


In [54]:
LAST=pd.concat([ide,LAST],axis=1)

In [55]:
LAST=pd.concat([ide.reset_index()['index'],LAST.reset_index().drop(['index'],axis=1)],axis=1) 

In [56]:
LAST=LAST.set_index('index')

In [57]:
LAST=LAST.sort_index()

In [58]:
LAST=LAST.drop(['level_0'],axis=1)

In [59]:
LAST

,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,...,['ul_branch_cnt']_8,['ul_branch_cnt']_10,['ul_branch_cnt']_11,['ul_branch_cnt']_12,['ul_branch_cnt']_17,['ul_branch_cnt']_20,bus_age,ul_founders_cnt,ogrn_age,ul_capital_sum
index,,,,,,,,,,,,,,,,,,,,,
0,17.5,17.5,17.5,2.3,0.6,12.2,11.9,12.7,12.6,0.9,...,0,0,0,0,0,0,156,1,135,11.512935
1,15.6,15.7,15.6,2.3,0.6,13.0,10.8,12.9,7.7,0.9,...,0,0,0,0,0,0,216,1,138,11.512935
2,17.8,17.7,17.7,2.3,0.6,15.0,14.1,14.5,13.1,0.9,...,0,0,0,0,0,0,244,2,138,12.429220
3,14.5,14.3,14.3,2.3,0.6,12.3,11.4,11.8,7.7,0.9,...,0,0,0,0,0,0,272,2,139,9.210440
4,19.2,19.2,19.2,2.3,0.6,16.5,16.4,14.6,13.0,0.9,...,0,0,0,0,0,0,254,84,139,14.346491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,16.9,16.8,16.6,14.8,0.6,14.7,11.7,14.7,13.1,0.9,...,0,0,0,0,0,0,47,1,47,9.210440
196,15.6,15.7,15.6,2.3,0.6,13.0,10.8,12.9,7.7,0.9,...,0,0,0,0,0,0,283,1,140,0.000000
197,15.6,15.7,15.6,2.3,0.6,13.0,10.8,12.9,7.7,0.9,...,0,0,0,0,0,0,22,1,22,9.903538


# формирование ответа

In [ ]:
answ_df = test_df[['record_id']]

In [ ]:
answ_df['id'] = answ_df['record_id']

In [ ]:
answ_df.drop('record_id',axis=1, inplace=True)

In [ ]:
cutoff = train_df_newversion_Y.mean()

In [ ]:
answ_df['predict'] = (log_ElNet2.predict_proba(LAST)[:,1]>cutoff).astype(int)

In [ ]:
answ_df.to_csv('16-11-19-06.csv',index=False, sep=';')

In [ ]:
answ_df['predict'].sum()

In [ ]:
train_df_newversion_edited.columns